In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import torch

In [9]:
from datasets import load_dataset

data_down_dir = "/root/storage/nas/JH_server/2025/Synthetic_data/0_dataset"

dataset = load_dataset(f"{data_down_dir}/20_newsgroups")
# 예시 출력
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 11314
    })
    test: Dataset({
        features: ['text', 'label', 'label_text'],
        num_rows: 7532
    })
})
{'text': 'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.', 'label': 7, 'label_text': 'rec.autos'}


In [18]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir='/root/storage/nas/JH_server/cache/')

def preprocess(inputs):
    return tokenizer(inputs['text'], truncation=True, padding="max_length", max_length=128)

train_dataset = dataset["train"].map(preprocess, batched=True)
test_dataset = dataset["test"].map(preprocess, batched=True)

num_labels =  len(set(dataset["test"]["label"]))
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, cache_dir='/root/storage/nas/JH_server/cache/')

accuracy = evaluate.load("accuracy")

def compute_metrics(pred):
    logits, labels = pred
    preds = np.argmax(logits, axis=1)
    acc = accuracy.compute(predictions=preds, references=labels)

    return acc

args = TrainingArguments(
    eval_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    save_strategy="no"
)

trainer = Trainer(
    model = model,
    args=args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

                                    

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_318725/4293970969.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss,Accuracy
100,No log,1.308403,0.641264
200,No log,1.124695,0.672464
300,No log,1.068857,0.684280
400,No log,1.047619,0.695964
500,1.017000,1.065817,0.696760
600,1.017000,1.086303,0.703133
700,1.017000,1.104061,0.703532
800,1.017000,1.177232,0.705656
900,1.017000,1.211919,0.705789
1000,0.253500,1.295017,0.700876


TrainOutput(global_step=1770, training_loss=0.40378716247903423, metrics={'train_runtime': 427.2759, 'train_samples_per_second': 264.794, 'train_steps_per_second': 4.143, 'total_flos': 7443298953277440.0, 'train_loss': 0.40378716247903423, 'epoch': 10.0})